# **Linear Regression Lab**
#### Лабораторная работа с сабсетом датасета [Million Song Dataset](http://labrosa.ee.columbia.edu/millionsong/) из [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD). 

#### Наша цель - обучить модель линейной регрессии для предсказания года выхода песни на основе аудио фичей данных песен
#### **Контент Лабораторной работы: **
+  ####*Part 1:* Загрузка данных
 + #### *Visualization 1:* Фичи
 + #### *Visualization 2:* Смещение таргета
+  ####*Part 2:* Создание базовой модели и ее оценка
 + #### *Visualization 3:* Predicted vs. actual
 + #### ** LinearModel and SGDRegressor
+  ####*Part 3:* Улучшение модели линейной регрессии и ее оценка



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [1]:
!gdown --id 1_cVtpGgUgbcUH4FaX_c8oN3L9yEyFnAK

Downloading...
From: https://drive.google.com/uc?id=1_cVtpGgUgbcUH4FaX_c8oN3L9yEyFnAK
To: /content/year_prediction.csv
443MB [00:06, 69.6MB/s]


### ** Part 1: Загрузка датасета **

In [ ]:
data = pd.read_csv('/content/year_prediction.csv', sep=',')

In [ ]:
# Посмотрите первые 5 наблюдений нашего датасета
data.head()

In [ ]:
# Определите размер датасета
data.shape

In [ ]:
# ДЛя целей обучения оставим только первые 12 признаков
data = data.iloc[:, :13]

In [ ]:
# переименуем label в year
data = data.rename(index=str, columns={"label":"year"})

#### **(1a) Исследуйте интервал наблюдений таргета **
#### самая первая песня датируется каким годом? самая последняя песня?

In [ ]:
min(data.year), max(data.year)

In [ ]:
nsongs = {}
for y in range(1922,2012):
    nsongs[y] = len(data[data.year==y])
yrs = range(1922,2011)
values = [nsongs[y] for y in yrs]
plt.bar(yrs, values, align='center')
plt.xlabel("Year")
plt.ylabel("Number of songs")

In [ ]:
# Разбейте датасет на тренировочный и валидационный путем выбора первых 463715 строк и определив их в тренировочный 
# И остальные строки в валидационный
# Возьмем только сэмпл из 60 тыс. этих строк 
X = data.iloc[:,:]
Y = data.iloc[:,0]

# Train set
train = X.iloc[0:463715,:].sample(60000, random_state = 42)
X_train = train.iloc[:, 1:]
y_train = X_train.iloc[:, 0]

In [ ]:
valid = X.iloc[463715:, :].sample(10000, random_state = 42)

In [ ]:
X_valid = valid.iloc[:, 1:]
y_valid = X_valid.iloc[:, 0]

In [ ]:
# Убедитесь, что измерения ваших датасетов соответствуют
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

In [ ]:
X_valid.sample(3)


### **Visualization 1 Heat Map: Фичи**
#### Загрузим библитеку для визауализации seaborn. 

#### Исследуем 50 случайных наблюдений с помощью heat map. С помощью этого графика мы можем наблюдать дисперсию каждого параметра в зависимости от цвета по шкале 

In [ ]:
# Возьмите 50 рандомных наблюдений и визуализируйте 

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(data.sample(50).iloc[:, 1:13])
plt.text(4.7, -3, 'Feature', color='#999999', size='11')

#### **(1d) Смещение меток **
#### Как мы видели ранее, метки находятся в промежутке 1900-х и 2000-х годов. На практике, часто имеет смысл сместить метки таргетов к нулю. Сместите значения таргета к нулю, чтобы год самой первой песни был равен нулю. Визуализируйте частотность по каждому новому значению метки (label)

In [ ]:
data.year = [i-1922 for i in data.year]

In [ ]:
# nsongs = {}
# for y in range(0,88):
#     nsongs[y] = len(X_train[X_train.year==y])
# yrs = range(0,88)
# values = [nsongs[y] for y in yrs]
# plt.bar(yrs, values, align='center')
# plt.xlabel("Year")
# plt.ylabel("Number of songs")

#### ** Visualization 2: Shifting labels **
#### Наши оба графика со смещением и изначальные данные по нашемы таргету должны выглядеть одинаково

In [ ]:
nsongs = {}
for y in range(0,89):
    nsongs[y] = len(data[data.year==y])
yrs = range(0,89)
values = [nsongs[y] for y in yrs]
plt.bar(yrs, values, align='center')
plt.xlabel("Year")
plt.ylabel("Number of songs")

### ** Part 2: Создание baseline модели и ее оценка **

#### **(2a) Среднее по таргету **
#### Наипростейшая модель baseline - это та модель, в которой мы делаем одинаковое значение предсказания независимо от параметров наблюдения, используя среднее в обучающей выборке, как константное значение предсказания. Посчитайте среднее по году выпуска песни в обучающей выборке 

In [ ]:
data.year.describe(), np.mean(data.year)

In [ ]:
from sklearn import model_selection, linear_model, metrics

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = linear_model.LinearRegression()
lm.fit(X_train, y_train)

In [ ]:
lm_predictions = lm.predict(X_valid)

In [ ]:
metrics.mean_absolute_error(y_valid, lm_predictions)

In [ ]:
calcRMSE(y_valid, lm_predictions)

#### **(2b) RMSE (Root mean squared error) **
#### Мы хотим посмотреть перфоманс этой наивной моедли. Будем использовать RMSE ([RMSE](http://en.wikipedia.org/wiki/Root-mean-square_deviation)) для оценочных целей. Напишите функцию для подсчета RMSE  для 1-го наблюдения из датасета

In [ ]:
import math


def squaredError(label, prediction):
    """Calculates the the squared error for a single prediction.

    Args:
        label (float): The correct value for this observation.
        prediction (float): The predicted value for this observation.

    Returns:
        float: The difference between the `label` and `prediction` squared.
    """
    return float((label - prediction)**2)

def calcRMSE(label_vector, prediction_vector):
    """Calculates the root mean squared error for an `RDD` of (label, prediction) tuples.

    Args:
        labelsAndPred (RDD of (float, float)): An `RDD` consisting of (label, prediction) tuples.

    Returns:
        float: The square root of the mean of the squared errors.
    """
    MSE =  sum([squaredError(x, y) for x, y in zip(label_vector, prediction_vector)])/len(label_vector)
    return  math.sqrt(MSE)


#### **(2c) RMSE обучающей, валидационной и тестовой выборок **
#### Давайте подсчитаем train, validation, test RMSE нашей базовой модели (baseline model). Используйте функцию calcRMSE. Заметьте, что каждый RMSE может интерпретироваться как средняя предиктивная ошибка для каждой из этих выборок

In [ ]:
label_train, prediction_train = y_train, np.repeat(np.mean(data.year), len(y_train))

In [ ]:
rmseTrainBase = calcRMSE(label_train, prediction_train)

In [ ]:
label_valid, prediction_valid = y_valid, np.repeat(np.mean(data.year), len(y_valid))

In [ ]:
rmseValidBase = calcRMSE(label_valid, prediction_valid)

#### ** Visualization 3: Predicted vs. actual **
#### Визуализируем предсказания валидационной выборки. scatter plots ниже визуализируют 1) идеальную ситуацию, когда значение предсказания и настоящия значения равны 2) значения настощяие vs. значения предсказаныне базовой моделью  

In [ ]:
from matplotlib.colors import ListedColormap, Normalize
from matplotlib.cm import get_cmap
cmap = get_cmap('YlOrRd')
norm = Normalize()

In [ ]:
plt.scatter(y_valid, y_valid, s=4**2, cmap = cmap, edgecolors='#888888', alpha=0.75, linewidths=0.5)
plt.xlabel('Actual')
plt.ylabel('Ideal Actual')
pass

In [ ]:
plt.scatter(regressor2_predictions, y_valid, s=4**2, cmap = cmap, edgecolors='#888888', alpha=0.75, linewidths=0.5)
plt.xlabel('Actual')
plt.ylabel('Ideal Actual')
pass

In [ ]:
plt.scatter(prediction_valid, label_valid, s=4**2, cmap = cmap, edgecolors='#888888', alpha=0.75, linewidths=0.5)
plt.xlabel('Actual')
plt.ylabel('Ideal Actual')
pass

#### Улучшение линейной модели

In [ ]:
# Нужно нормализовать данные 
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)  # Don't cheat - fit only on training data
X_train_wo = scaler.transform(X_train)
X_valid_wo = scaler.transform(X_valid) 

In [ ]:
X_train_wo.shape

In [ ]:
X_test_wo.shape

In [ ]:
from sklearn import model_selection, linear_model, metrics, pipeline, preprocessing

In [ ]:
regressor = linear_model.SGDRegressor(random_state = 0, loss = 'squared_loss', penalty = 'l2')
regressor.fit(X_train_wo, y_train)

In [ ]:
regressor.fit(X_train_wo, y_train)

In [ ]:
sgd_predictions = regressor.predict(X_valid_wo)

In [ ]:
metrics.mean_absolute_error(y_valid, sgd_predictions)

In [ ]:
regressor2 = linear_model.LinearRegression()

In [ ]:
regressor2.fit(X_train_wo, y_train)

In [ ]:
regressor2_predictions = regressor2.predict(X_valid_wo)

In [ ]:
metrics.mean_absolute_error(y_valid, regressor2_predictions)

In [ ]:
metrics.mean_absolute_error(y_valid, lm_predictions) - metrics.mean_absolute_error(y_valid, regressor2_predictions)

In [ ]:
calcRMSE(y_valid, regressor2_predictions) - calcRMSE(y_valid, lm_predictions)

#### ** По итогу **
#### Загрузили данные
#### Изучили распределение таргет и взаимосвязи между фичами
#### Мы разбили наши данные на обучающую и тестовую выборки
#### 
#### Мы обучили модель регрессии на 60 тыс. строках нашего тренировочной выборки.
#### Оценили то, как она работает на валидацонной выборке
#### Применили функцию  calcRMSE для оценки работы линейной регрессии
#### Сделали прогнозы на нашей тестовой выборке
